In [1]:
import random
import numpy as np

In [2]:
class GridWorld():
    def __init__(self):
        self.x=0
        self.y=0
    
    def step(self, a):
        if a==0:
            self.move_left()
        elif a==1:
            self.move_up()
        elif a==2:
            self.move_right()
        elif a==3:
            self.move_down()

        reward = -1  
        done = self.is_done()
        return (self.x, self.y), reward, done

    def move_left(self):
        if self.y==0:
            pass
        elif self.y==3 and self.x in [0,1,2]:
            pass
        elif self.y==5 and self.x in [2,3,4]:
            pass
        else:
            self.y -= 1

    def move_right(self):
        if self.y==1 and self.x in [0,1,2]:
            pass
        elif self.y==3 and self.x in [2,3,4]:
            pass
        elif self.y==6:
            pass
        else:
            self.y += 1
      
    def move_up(self):
        if self.x==0:
            pass
        elif self.x==3 and self.y==2:
            pass
        else:
            self.x -= 1

    def move_down(self):
        if self.x==4:
            pass
        elif self.x==1 and self.y==4:
            pass
        else:
            self.x+=1

    def is_done(self):
        if self.x==4 and self.y==6:
            return True
        else:
            return False
      
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

In [ ]:
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) # 각 state마다 action table
        self.eps = 0.9  # 입실론 : 0.9로 시작
        self.alpha = 0.01
        self.gamma = 1
        
    def select_action(self, s):
        x, y = s
        coin = random.random()
        if coin < self.eps: # 입실론보다 작으면 랜덤하게 선택
            action = random.randint(0,3)
        else:
            action_val = self.q_table[x,y,:] # 입실론보다 크면 
            action = np.argmax(action_val) # 현재 state의 action에서 가장 큰 값 선택
        
        return action

    def update_table(self, history): # history를 받음
        cum_reward = 0 # 리턴
        for transition in history[::-1]: # 뒤에서부터
            s, a, r, s_prime = transition # state, action, reward, next_state
            x,y = s # 현재 state
            self.q_table[x,y,a] = self.q_table[x,y,a] + self.alpha * (cum_reward - self.q_table[x,y,a]) # q_table 업데이트
            cum_reward = r + self.gamma*cum_reward

    def anneal_eps(self): # decaying epsilon
        self.eps -= 0.03
        self.eps = max(self.eps, 0.1)

    def show_table(self):
        q_lst = self.q_table.tolist() # shape = (5,7,4)
        data = np.zeros((5,7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

In [ ]:
def main():
    env = GridWorld()
    agent = QAgent()
    epi = 100000
    for n_epi in range(epi): # episode
        done = False
        history = []

        s = env.reset()
        while not done: # 에이전트 경험 쌓기
            a = agent.select_action(s) # 주어진 state에서 action
            s_prime, r, done = env.step(a) # next_state, reward, 종료여부 반환
            history.append((s, a, r, s_prime))
            s = s_prime
        agent.update_table(history) # Q_table 업데이트
        agent.anneal_eps() # decaying epsilon

    agent.show_table()

In [ ]:
main()

[[2. 3. 0. 2. 2. 2. 3.]
 [2. 3. 0. 2. 2. 3. 3.]
 [2. 3. 0. 1. 0. 3. 3.]
 [2. 2. 2. 1. 0. 3. 3.]
 [1. 1. 2. 1. 0. 2. 0.]]
